<a href="https://colab.research.google.com/github/Nachi2006/NeuralNet-IOS-Implementation/blob/main/NNCML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# This script demonstrates how to create a PyTorch model and convert it to a Core ML .mlpackage file.

# First, make sure you have the necessary libraries installed.
# pip install torch coremltools scikit-learn numpy

import torch
import torch.nn as nn
import coremltools as ct
import warnings
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import SGD
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
warnings.filterwarnings("ignore", category=UserWarning)
#Define Model
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)  # Input layer with 4 features, hidden layer with 16 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)   # Hidden layer to output layer with 3 classes

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
#Set Model to Evaluation Mode
model = IrisNet()
model.eval()
# Dataset Loading
print("Loading and preparing the Iris dataset...")
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Tensor Conversion
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#Data Loading
dataset = TensorDataset(X_train_tensor, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
optimizer = SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
#Model Training
print("Training the model...")
for epoch in range(50):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
print("Training complete.")
print("-" * 30)
#Dummy Inputs for Model Tracing
dummy_input = torch.tensor(X_test[0:1], dtype=torch.float32)
#Model Tracing
traced_model = torch.jit.trace(model, dummy_input)
#Model Conversion
coreml_model = ct.convert(
    traced_model,
    inputs=[ct.TensorType(shape=dummy_input.shape)]
)
#Save the Core ML model to a file.
coreml_model.save("iris_net.mlpackage")

print("PyTorch model successfully converted and saved as iris_net.mlpackage")
print("-" * 30)



Loading and preparing the Iris dataset...
Training the model...


Training complete.
------------------------------


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 3695.96 passes/s]


PyTorch model successfully converted and saved as iris_net.mlpackage
------------------------------
